In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib as plt

!git clone 'https://github.com/MFSJMenger/python_lessons.git'


Cloning into 'python_lessons'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 25 (delta 4), reused 20 (delta 2), pack-reused 0
Unpacking objects: 100% (25/25), done.


Import mass libary from the file. 
Reminder: the value in the mass file (MASSES.json) is in the unit of electron rest mass ($m_e$). 

use formular to convert $m_e$ to atomic mass unit $a.m.u$: 

$$
 1 m_e = 5.485 \times 10^{-4} a.m.u.
$$


In [2]:
import json
# import mass libary from the file 
with open('python_lessons/assignment3/MASSES.json', 'r') as f:
  MASSES = json.load(f)

MASSES = {int(key): value for key, value in MASSES.items()} 

# creat a dictionary with amu for the first 18 elements in the peordic table

element_name = ['H','He','Li','Be','B','C','N','O','F','Ne','Na','Mg','Al','Si','P','S','Cl','Ar']

element_lib = dict(zip(range(18), element_name))

mass_lib = {element_lib[key-1]: value*5.485e-4 for key, value in list (MASSES.items())[:18]}
mass_lib

{'Al': 26.977607707831,
 'Ar': 39.956561840325584,
 'B': 11.007701318800372,
 'Be': 9.010869231678928,
 'C': 11.998252008242524,
 'Cl': 34.96375922776564,
 'F': 18.995635579012568,
 'H': 1.0076781941839186,
 'He': 4.002019956912296,
 'Li': 7.014982006903132,
 'Mg': 23.981548195356776,
 'N': 14.001034228505723,
 'Na': 22.986421172627814,
 'Ne': 19.989527781639012,
 'O': 15.992585085034875,
 'P': 30.9692501766105,
 'S': 31.96741375695188,
 'Si': 27.972851713517045}

# Normal modes of H2O


### read data from files

In [3]:
### store the number of atoms
with open('python_lessons/assignment3/h2o.xyz', 'r') as f:
  natom = int(f.readline())
print(natom)

#get the atom names from the file and store it into a list.  
with open('python_lessons/assignment3/h2o.xyz', 'r') as f:
  next(f)
  next(f)
  atoms_list=[item[0] for item in f.readlines()]

print(atoms_list)


3
['O', 'H', 'H']


### mass weight the hessian

$F_{ij}^M$ = $F_{ij}$/$\sqrt{(m_i \times m_j)}$

- step1: $F_{ij}$ is read from "hessian.dat" file and stored as a numpy array.

- step2: creat another matrix ( here G), and store the $1/\sqrt{(m_i \times m_j)}$ into it.

- step3: multply the two matrix to get massweighted hessian.


In [4]:
### Read hessian data from file and store it as a numpy array
'''
An example of "how to open files and store it as a numpy array"
by Max show during the lecture 
'''

with open('python_lessons/assignment3/h2o_hessian.dat', 'r') as f:
  data_h2o = np.array([float(value) for line in f.readlines()[1:] for value in line.split()])
data_h2o

array([ 0.09276434,  0.        ,  0.        , -0.04638217,  0.        ,
        0.        , -0.04638217,  0.        ,  0.        ,  0.        ,
        0.31713271,  0.        ,  0.        , -0.15856636,  0.0800202 ,
        0.        , -0.15856636, -0.0800202 ,  0.        ,  0.        ,
        0.28009073,  0.        ,  0.03477659, -0.14004536,  0.        ,
       -0.03477659, -0.14004536, -0.04638217,  0.        ,  0.        ,
        0.05146682,  0.        ,  0.        , -0.00508465,  0.        ,
        0.        ,  0.        , -0.15856636,  0.03477659,  0.        ,
        0.17300755, -0.05739839,  0.        , -0.0144412 ,  0.02262181,
        0.        ,  0.0800202 , -0.14004536,  0.        , -0.05739839,
        0.12683735,  0.        , -0.02262181,  0.01320802, -0.04638217,
        0.        ,  0.        , -0.00508465,  0.        ,  0.        ,
        0.05146682,  0.        ,  0.        ,  0.        , -0.15856636,
       -0.03477659,  0.        , -0.0144412 , -0.02262181,  0.  

In [5]:
### Reshape the numpy array to a squared matrix: (3N)^2 
hessian_h2o = data_h2o.reshape((3*natom,3*natom))
hessian_h2o
# print(hessian_h2o[0])
# print(hessian_h2o[0,:3])

array([[ 0.09276434,  0.        ,  0.        , -0.04638217,  0.        ,
         0.        , -0.04638217,  0.        ,  0.        ],
       [ 0.        ,  0.31713271,  0.        ,  0.        , -0.15856636,
         0.0800202 ,  0.        , -0.15856636, -0.0800202 ],
       [ 0.        ,  0.        ,  0.28009073,  0.        ,  0.03477659,
        -0.14004536,  0.        , -0.03477659, -0.14004536],
       [-0.04638217,  0.        ,  0.        ,  0.05146682,  0.        ,
         0.        , -0.00508465,  0.        ,  0.        ],
       [ 0.        , -0.15856636,  0.03477659,  0.        ,  0.17300755,
        -0.05739839,  0.        , -0.0144412 ,  0.02262181],
       [ 0.        ,  0.0800202 , -0.14004536,  0.        , -0.05739839,
         0.12683735,  0.        , -0.02262181,  0.01320802],
       [-0.04638217,  0.        ,  0.        , -0.00508465,  0.        ,
         0.        ,  0.05146682,  0.        ,  0.        ],
       [ 0.        , -0.15856636, -0.03477659,  0.        , -0

In [6]:
### creat a G matrix
G_matrix = hessian_h2o.copy() #This is to make sure G matrix has the same shape as Hessian matrix, values will be overwritten.
for i in range(natom*3):
  j=0
  if i < 3:
    G_matrix[i,:j+3]=(1/np.sqrt(mass_lib[atoms_list[j]]*mass_lib[atoms_list[j]]))
    G_matrix[i,j+3:j+6]=(1/np.sqrt(mass_lib[atoms_list[j]]*mass_lib[atoms_list[j+1]]))
    G_matrix[i,j+6:j+9]=(1/np.sqrt(mass_lib[atoms_list[j]]*mass_lib[atoms_list[j+2]]))
  if i>=3 and i<6:
    G_matrix[i,:j+3]=(1/np.sqrt(mass_lib[atoms_list[j+1]]*mass_lib[atoms_list[j]]))
    G_matrix[i,j+3:j+6]=(1/np.sqrt(mass_lib[atoms_list[j+1]]*mass_lib[atoms_list[j+1]]))
    G_matrix[i,j+6:j+9]=(1/np.sqrt(mass_lib[atoms_list[j+1]]*mass_lib[atoms_list[j+2]]))
  if i>=6 and i<9:
    G_matrix[i,:j+3]=(1/np.sqrt(mass_lib[atoms_list[j+2]]*mass_lib[atoms_list[j]]))
    G_matrix[i,j+3:j+6]=(1/np.sqrt(mass_lib[atoms_list[j+2]]*mass_lib[atoms_list[j+1]]))
    G_matrix[i,j+6:j+9]=(1/np.sqrt(mass_lib[atoms_list[j+2]]*mass_lib[atoms_list[j+2]]))

G_matrix

array([[0.06252898, 0.06252898, 0.06252898, 0.24910345, 0.24910345,
        0.24910345, 0.24910345, 0.24910345, 0.24910345],
       [0.06252898, 0.06252898, 0.06252898, 0.24910345, 0.24910345,
        0.24910345, 0.24910345, 0.24910345, 0.24910345],
       [0.06252898, 0.06252898, 0.06252898, 0.24910345, 0.24910345,
        0.24910345, 0.24910345, 0.24910345, 0.24910345],
       [0.24910345, 0.24910345, 0.24910345, 0.99238031, 0.99238031,
        0.99238031, 0.99238031, 0.99238031, 0.99238031],
       [0.24910345, 0.24910345, 0.24910345, 0.99238031, 0.99238031,
        0.99238031, 0.99238031, 0.99238031, 0.99238031],
       [0.24910345, 0.24910345, 0.24910345, 0.99238031, 0.99238031,
        0.99238031, 0.99238031, 0.99238031, 0.99238031],
       [0.24910345, 0.24910345, 0.24910345, 0.99238031, 0.99238031,
        0.99238031, 0.99238031, 0.99238031, 0.99238031],
       [0.24910345, 0.24910345, 0.24910345, 0.99238031, 0.99238031,
        0.99238031, 0.99238031, 0.99238031, 0.99238031],


In [7]:
G_matrix_v1 = hessian_h2o.copy()
i=0
j=0
for i in range(natom):
  for j in range(natom):
    G_matrix_v1[i*3,j*3]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix_v1[i*3,j*3+1]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix_v1[i*3,j*3+2]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix_v1[i*3+1,j*3]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix_v1[i*3+1,j*3+1]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix_v1[i*3+1,j*3+2]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix_v1[i*3+2,j*3]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix_v1[i*3+2,j*3+1]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix_v1[i*3+2,j*3+2]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    # print("i value:" + str(i))
    # print("J value:" + str(j))

G_matrix_v1

array([[0.06252898, 0.06252898, 0.06252898, 0.24910345, 0.24910345,
        0.24910345, 0.24910345, 0.24910345, 0.24910345],
       [0.06252898, 0.06252898, 0.06252898, 0.24910345, 0.24910345,
        0.24910345, 0.24910345, 0.24910345, 0.24910345],
       [0.06252898, 0.06252898, 0.06252898, 0.24910345, 0.24910345,
        0.24910345, 0.24910345, 0.24910345, 0.24910345],
       [0.24910345, 0.24910345, 0.24910345, 0.99238031, 0.99238031,
        0.99238031, 0.99238031, 0.99238031, 0.99238031],
       [0.24910345, 0.24910345, 0.24910345, 0.99238031, 0.99238031,
        0.99238031, 0.99238031, 0.99238031, 0.99238031],
       [0.24910345, 0.24910345, 0.24910345, 0.99238031, 0.99238031,
        0.99238031, 0.99238031, 0.99238031, 0.99238031],
       [0.24910345, 0.24910345, 0.24910345, 0.99238031, 0.99238031,
        0.99238031, 0.99238031, 0.99238031, 0.99238031],
       [0.24910345, 0.24910345, 0.24910345, 0.99238031, 0.99238031,
        0.99238031, 0.99238031, 0.99238031, 0.99238031],


In [8]:
### multiply the hessian matrix with G matrix to get massweighed hessian matrix
massweighed_hessian = np.multiply(hessian_h2o,G_matrix)
print(massweighed_hessian)

[[ 0.00580046  0.          0.         -0.01155396  0.          0.
  -0.01155396  0.          0.        ]
 [ 0.          0.01982998  0.          0.         -0.03949943  0.01993331
   0.         -0.03949943 -0.01993331]
 [ 0.          0.          0.01751379  0.          0.00866297 -0.03488578
   0.         -0.00866297 -0.03488578]
 [-0.01155396  0.          0.          0.05107466  0.          0.
  -0.00504591  0.          0.        ]
 [ 0.         -0.03949943  0.00866297  0.          0.17168929 -0.05696104
   0.         -0.01433116  0.02244944]
 [ 0.          0.01993331 -0.03488578  0.         -0.05696104  0.12587089
   0.         -0.02244944  0.01310737]
 [-0.01155396  0.          0.         -0.00504591  0.          0.
   0.05107466  0.          0.        ]
 [ 0.         -0.03949943 -0.00866297  0.         -0.01433116 -0.02244944
   0.          0.17168929  0.05696104]
 [ 0.         -0.01993331 -0.03488578  0.          0.02244944  0.01310737
   0.          0.05696104  0.12587089]]


### diagnolize the massweighed hessian matrix

$F^M$ $\times$ $L$ = $L$ $\times$ $\Lambda$

At the diagonal of matrix $\Lambda$ are values of ${\lambda}_i$, which is useful.

In [11]:
eigvalue, eigvector = np.linalg.eig(massweighed_hessian)
# print(eigvalue)
# print(eigvector)

# eig_format=[abs(float('%.10f'%item)) for item in eigvalue]  #becasue there are negative values
eig_format= sorted(eig_format, reverse=True)
eig_format


[0.2351885029,
 0.210742019,
 0.1317704776,
 0.0561205722,
 0.0547631247,
 0.0518292111,
 0.0,
 0.0,
 0.0]

### compute the harmonic vibrational frequencies

<!-- For Simple harmonic motion involve a mass, $m$, and force constant $k$, the frequency $f$, in cycles per second is given by:

$f$ = $1/(2\pi) \times \sqrt{k/m}$ cycles per second

But, by convention, vibrationa frequencies in UV-Visible spectroscopy are reported in reciprocal centimeters. To convert from $f$ to $v$ (or in our case $w_i$), $f$ must be divided by the speed of light, $c$. -->

$w_i = f/c = 1/(2\pi c) \times (k/m)^{1/2} = 1/(2\pi c) \times (\lambda_{i})^{1/2}    cm^{-1}$ 

$w_i$ = constant $\times$ $\sqrt{{\lambda}_i}$

We need correct units. 

$\lambda$ unit is: $ E_h /( a.m.u \times Bohr^2)$  

Hartree atomic units:
- bohr radius: $B =1$ or $ a_0 =1 $, atomic unit of length
- electron mass: $ m_e = 1 $,  atomic unit of mass
- reduced planck constant: $\bar{h} = 1$, atomic unit of action
- energy $ E_h = \bar{h}^2 /(m_e a_0)$




convert atomic mass unit to atomic unit: $1a.m.u. = 1822.888 a.u.$

In the equation, $E_h$ and $amu$ are both now in atomic unit. 

$bohr$ convert to cm :  $  5.29 \times 10^{-9} cm $

Use c also in atomic unit: 137.036 a.u. 

Then the final unit should be $cm^{-1}$
 

In [12]:
amu_to_au = 1/(5.485e-4)
bohr_to_cm = 5.291e-9
c_in_au = 137.036
 
CONSTANT = 1/(2*np.pi*c_in_au) * np.sqrt(1/(amu_to_au * bohr_to_cm * bohr_to_cm))

print(CONSTANT)

frequency = CONSTANT * np.sqrt(eig_format)
frequency

5140.862790614337


array([2493.12501648, 2359.99770673, 1866.14365596, 1217.85913462,
       1203.0401535 , 1170.37035279,    0.        ,    0.        ,
          0.        ])

# Normal modes for 3C1b


### read data from files

In [13]:
with open('python_lessons/assignment3/3c1b.xyz', 'r') as f:
  natom = int(f.readline())
# print(data_xyz)

### store the number of atoms
# natom = int(data_xyz[])
print(natom)

#get the atom names from the file and store it into a list.  
with open('python_lessons/assignment3/3c1b.xyz', 'r') as f:
  next(f) #skip the 1st line
  next(f) #skip the 2nd line
  atoms_list=[item[1:3] for item in f.readlines()]  # because it needs to read 2 character in the line.

atoms_list = [item.strip() for item in atoms_list]
print(atoms_list)   

12
['C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'Cl']


### mass wight the hessian

In [14]:
### Read hessian data from file and store it as a numpy array

with open('python_lessons/assignment3/3c1b_hessian.dat', 'r') as f:
  data_3c1b = np.array([float(value) for line in f.readlines()[1:] for value in line.split()])
data_3c1b

array([ 0.86578654, -0.01937235,  0.00372856, ...,  0.06292554,
        0.02784126,  0.0490802 ])

In [15]:
### Reshape the numpy array to a squared matrix: (3N)^2 
hessian_3c1b = data_3c1b.reshape((3*natom,3*natom))
print(hessian_3c1b.shape)
print(hessian_3c1b)

(36, 36)
[[ 0.86578654 -0.01937235  0.00372856 ... -0.0186761  -0.03655251
   0.00733018]
 [-0.01937235  0.7176365   0.05414804 ... -0.00705031 -0.01355535
   0.00337426]
 [ 0.00372856  0.05414804  0.84954439 ... -0.00688474 -0.01221798
   0.00513001]
 ...
 [-0.0186761  -0.00705031 -0.00688474 ...  0.26124074  0.09371806
   0.06292554]
 [-0.03655251 -0.01355535 -0.01221798 ...  0.09371806  0.07997372
   0.02784126]
 [ 0.00733018  0.00337426  0.00513001 ...  0.06292554  0.02784126
   0.0490802 ]]


In [16]:
### creat a G matrix
G_matrix = hessian_3c1b.copy() #This is to make sure G matrix has the same shape as Hessian matrix, values will be overwritten.

for i in range(natom):
  for j in range(natom):
    G_matrix[i*3,j*3]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3,j*3+1]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3,j*3+2]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+1,j*3]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+1,j*3+1]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+1,j*3+2]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+2,j*3]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+2,j*3+1]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+2,j*3+2]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))

print(G_matrix.shape)
G_matrix

(36, 36)


array([[0.08334547, 0.08334547, 0.08334547, ..., 0.04882384, 0.04882384,
        0.04882384],
       [0.08334547, 0.08334547, 0.08334547, ..., 0.04882384, 0.04882384,
        0.04882384],
       [0.08334547, 0.08334547, 0.08334547, ..., 0.04882384, 0.04882384,
        0.04882384],
       ...,
       [0.04882384, 0.04882384, 0.04882384, ..., 0.02860104, 0.02860104,
        0.02860104],
       [0.04882384, 0.04882384, 0.04882384, ..., 0.02860104, 0.02860104,
        0.02860104],
       [0.04882384, 0.04882384, 0.04882384, ..., 0.02860104, 0.02860104,
        0.02860104]])

In [17]:
### multiply the hessian matrix with G matrix to get massweighed hessian matrix
massweighed_hessian = np.multiply(hessian_3c1b,G_matrix)
print(massweighed_hessian)

[[ 0.07215939 -0.0016146   0.00031076 ... -0.00091184 -0.00178463
   0.00035789]
 [-0.0016146   0.05981175  0.00451299 ... -0.00034422 -0.00066182
   0.00016474]
 [ 0.00031076  0.00451299  0.07080568 ... -0.00033614 -0.00059653
   0.00025047]
 ...
 [-0.00091184 -0.00034422 -0.00033614 ...  0.00747176  0.00268043
   0.00179974]
 [-0.00178463 -0.00066182 -0.00059653 ...  0.00268043  0.00228733
   0.00079629]
 [ 0.00035789  0.00016474  0.00025047 ...  0.00179974  0.00079629
   0.00140375]]


### diagnolize the massweighed Hessian matrix

In [18]:
eigvalue, eigvector = np.linalg.eig(massweighed_hessian)
# print(eigvalue)
# print(eigvector)

# eig_format=[abs(float('%.10f'%item)) for item in eigvalue]  #becasue there are negative values
eig_format= sorted(eigvalue, reverse=True)
eig_format

[0.550499627862364,
 0.5373313412731113,
 0.5351964409991333,
 0.5256496004223138,
 0.5063704216833923,
 0.49933032867440563,
 0.4845897103280118,
 0.15949105907935068,
 0.12463888345718861,
 0.12408088818412674,
 0.11245535679757149,
 0.11023441656751536,
 0.09405692344139108,
 0.08979781035539734,
 0.08361222920043489,
 0.0690695975346926,
 0.06408134680757241,
 0.05959095636874076,
 0.05700052806981411,
 0.051215942786097125,
 0.049898137929709416,
 0.0410396879412218,
 0.02838715704026697,
 0.02137536202362542,
 0.008885006750759187,
 0.006529843507619276,
 0.004645054664144892,
 0.0028074141884905724,
 0.0022841147793415425,
 1.154905617557968e-07,
 8.709761999944657e-08,
 5.088786290099255e-09,
 3.524253289467955e-11,
 3.19384928442828e-11,
 3.7397384358695465e-12,
 -0.0004144898858780982]

### compute the harmonic vibrational frequencies

In [19]:
amu_to_au = 1/(5.485e-4)
bohr_to_cm = 5.291e-9
c_in_au = 137.036
 
CONSTANT = 1/(2*np.pi*c_in_au) * np.sqrt(1/(amu_to_au * bohr_to_cm * bohr_to_cm))

print(CONSTANT)

frequency = CONSTANT * np.sqrt(eig_format)
frequency

5140.862790614337


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in sqrt
  if __name__ == '__main__':


array([3.81429719e+03, 3.76840090e+03, 3.76090723e+03, 3.72721274e+03,
       3.65822301e+03, 3.63270378e+03, 3.57868198e+03, 2.05307202e+03,
       1.81494215e+03, 1.81087494e+03, 1.72395564e+03, 1.70684709e+03,
       1.57663585e+03, 1.54052548e+03, 1.48652054e+03, 1.35107505e+03,
       1.30137311e+03, 1.25494933e+03, 1.22736985e+03, 1.16342555e+03,
       1.14836033e+03, 1.04144907e+03, 8.66157685e+02, 7.51610350e+02,
       4.84579340e+02, 4.15419998e+02, 3.50373642e+02, 2.72388807e+02,
       2.45694240e+02, 1.74706579e+00, 1.51718715e+00, 3.66727197e-01,
       3.05189487e-02, 2.90531497e-02, 9.94160781e-03,            nan])

# Normal modes for Benzene

### read data from files

In [ ]:
with open('python_lessons/assignment3/benzene.xyz', 'r') as f:
  natom = int(f.readline())
  data_xyz= f.read()
data_xyz

### store the number of atoms
# natom = int(data_xyz[])
print(natom)

#get the atom names from the file and store it into a list.  
with open('python_lessons/assignment3/benzene.xyz', 'r') as f:
  next(f) #skip the 1st line
  next(f) #skip the 2nd line
  atoms_list=[item[0] for item in f.readlines()]

print(atoms_list)

12
['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H']


### mass weight the hessian

In [ ]:
### Read hessian data from file and store it as a numpy array

with open('python_lessons/assignment3/benzene_hessian.dat', 'r') as f:
  data_benzene = np.array([float(value) for line in f.readlines()[1:] for value in line.split()])
data_benzene

array([0.83704759, 0.        , 0.        , ..., 0.        , 0.        ,
       0.03685694])

In [ ]:
### Reshape the numpy array to a squared matrix: (3N)^2 
hessian_benzene= data_benzene.reshape((3*natom,3*natom))
hessian_benzene


array([[ 0.83704759,  0.        ,  0.        , ..., -0.08149339,
         0.        ,  0.        ],
       [ 0.        ,  1.02782495,  0.        , ...,  0.        ,
        -0.47489408,  0.        ],
       [ 0.        ,  0.        ,  0.1884039 , ...,  0.        ,
         0.        , -0.05333218],
       ...,
       [-0.08149339,  0.        ,  0.        , ...,  0.07696693,
         0.        ,  0.        ],
       [ 0.        , -0.47489408,  0.        , ...,  0.        ,
         0.48721616,  0.        ],
       [ 0.        ,  0.        , -0.05333218, ...,  0.        ,
         0.        ,  0.03685694]])

In [ ]:
### creat a G matrix
G_matrix = hessian_benzene.copy() #This is to make sure G matrix has the same shape as Hessian matrix, values will be overwritten.
 
for i in range(natom):
  for j in range(natom):
    G_matrix[i*3,j*3]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3,j*3+1]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3,j*3+2]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+1,j*3]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+1,j*3+1]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+1,j*3+2]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+2,j*3]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+2,j*3+1]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+2,j*3+2]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))

print(G_matrix.shape)
G_matrix

(36, 36)


array([[0.08334547, 0.08334547, 0.08334547, ..., 0.28759417, 0.28759417,
        0.28759417],
       [0.08334547, 0.08334547, 0.08334547, ..., 0.28759417, 0.28759417,
        0.28759417],
       [0.08334547, 0.08334547, 0.08334547, ..., 0.28759417, 0.28759417,
        0.28759417],
       ...,
       [0.28759417, 0.28759417, 0.28759417, ..., 0.99238031, 0.99238031,
        0.99238031],
       [0.28759417, 0.28759417, 0.28759417, ..., 0.99238031, 0.99238031,
        0.99238031],
       [0.28759417, 0.28759417, 0.28759417, ..., 0.99238031, 0.99238031,
        0.99238031]])

In [ ]:
### multiply the hessian matrix with G matrix to get massweighed hessian matrix
massweighed_hessian = np.multiply(hessian_benzene,G_matrix)
print(massweighed_hessian)

[[ 0.06976413  0.          0.         ... -0.02343702  0.
   0.        ]
 [ 0.          0.08566456  0.         ...  0.         -0.13657677
   0.        ]
 [ 0.          0.          0.01570261 ...  0.          0.
  -0.01533802]
 ...
 [-0.02343702  0.          0.         ...  0.07638047  0.
   0.        ]
 [ 0.         -0.13657677  0.         ...  0.          0.48350373
   0.        ]
 [ 0.          0.         -0.01533802 ...  0.          0.
   0.0365761 ]]


### diagnolize the massweighed Hessian matrix

In [ ]:
eigvalue, eigvector = np.linalg.eig(massweighed_hessian)
# print(eigvalue)
# print(eigvector)

eig_format=[abs(float('%.10f'%item)) for item in eigvalue]  #becasue there are negative values
eig_format= sorted(eig_format, reverse=True)
eig_format

[0.5315127935,
 0.5284384053,
 0.5284384053,
 0.5245736867,
 0.5245736866,
 0.5193936703,
 0.1413443565,
 0.1413443565,
 0.1189236952,
 0.1189236951,
 0.0963232622,
 0.0717668965,
 0.0711782606,
 0.0711782605,
 0.0568770451,
 0.0568770451,
 0.0558454662,
 0.0536566162,
 0.0536566162,
 0.0520270903,
 0.0506425474,
 0.0504302154,
 0.0407648337,
 0.0407648337,
 0.026783813,
 0.0248987293,
 0.0185549362,
 0.0185549362,
 0.00865067,
 0.00865067,
 1.7e-08,
 1.69e-08,
 1.69e-08,
 0.0,
 0.0,
 0.0]

### compute the harmonic vibrational frequencies

In [ ]:
amu_to_au = 1/(5.485e-4)
bohr_to_cm = 5.291e-9
c_in_au = 137.036
 
CONSTANT = 1/(2*np.pi*c_in_au) * np.sqrt(1/(amu_to_au * bohr_to_cm * bohr_to_cm))

print(CONSTANT)

frequency = CONSTANT * np.sqrt(eig_format)
frequency

5140.862790614337


array([3.74794211e+03, 3.73708692e+03, 3.73708692e+03, 3.72339631e+03,
       3.72339631e+03, 3.70496696e+03, 1.93274807e+03, 1.93274807e+03,
       1.77284273e+03, 1.77284273e+03, 1.59551762e+03, 1.37720343e+03,
       1.37154385e+03, 1.37154385e+03, 1.22603968e+03, 1.22603968e+03,
       1.21487046e+03, 1.19082420e+03, 1.19082420e+03, 1.17260241e+03,
       1.15689457e+03, 1.15446674e+03, 1.03795578e+03, 1.03795578e+03,
       8.41341313e+02, 8.11193766e+02, 7.00270361e+02, 7.00270361e+02,
       4.78146395e+02, 4.78146395e+02, 6.70286501e-01, 6.68312163e-01,
       6.68312163e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

# Write universal scripts

Description: integrate each step together. Write a code that automatically does the following steps. 

1. read data from file
2. mass weighted the hessian
3. diagnolize the massweighed hessian
4. compute harmonic vibrational frequencies

Keep in mind:

natom; atoms_list, hessian_raw_dat, hessian_matrix, G_matrix, hessian_massweight, eigvalue, eigvector, eigvalue_format

In [ ]:
### def a function that read xyz data from files
def read_xyzfile(filename, prefix = 'python_lessons/assignment3/'):
  if prefix is not None:
    filename=f"{prefix}{filename}"
  with open(filename, "r") as f:
    natom = int(f.readline())
    molname = f.readline()
    atoms_list=[item[:3] for item in f.readlines()]
  atoms_list = [item.strip() for item in atoms_list]
  # return print(natom), print(molname),print(atoms_list)
  return natom, molname, atoms_list

# read_xyzfile("3c1b.xyz")

### def a function that reads hessian data 
def read_hessian(filename, prefix = 'python_lessons/assignments3/'):
  if prefix is not None:
    filename = f"{prefix}{filename}"
  with open(filename +"_hessian.dat", "r") as f:
    hessian_data = np.array([float(value) for line in f.readlines()[1:] for value in line.split()])
    hessian_matrix. = hessian_data.reshape(3*natom, 3*natom)
  return hessian_matrix

def Gmatrix(matrix, natom):
  G_matrix = matrix.copy() #This is to make sure G matrix has the same shape as Hessian matrix, values will be overwritten.
  for i in range(natom):
    for j in range(natom):
      G_matrix[i*3,j*3]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
      G_matrix[i*3,j*3+1]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
      G_matrix[i*3,j*3+2]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
      G_matrix[i*3+1,j*3]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
      G_matrix[i*3+1,j*3+1]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
      G_matrix[i*3+1,j*3+2]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
      G_matrix[i*3+2,j*3]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
      G_matrix[i*3+2,j*3+1]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
      G_matrix[i*3+2,j*3+2]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
  return G_matrix

def massweight_hessian(x, y):
  massweight_matrix = np.multiply(x,y)
  return massweight_matrix

def Cal_freq(x):
  eigvalue, eigvector = np.linalg.eig(massweighed_hessian)
  
### def a function that compute the final vibrational frequencies

(12, ' 3c1b \n', ['C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'Cl'])

In [ ]:
with open('python_lessons/assignment3/benzene.xyz', 'r') as f:
  natom = int(f.readline())
  data_xyz= f.read()
data_xyz

### store the number of atoms
# natom = int(data_xyz[])
print(natom)

#get the atom names from the file and store it into a list.  
with open('python_lessons/assignment3/benzene.xyz', 'r') as f:
  next(f) #skip the 1st line
  next(f) #skip the 2nd line
  atoms_list=[item[0] for item in f.readlines()]

print(atoms_list)

### Read hessian data from file and store it as a numpy array

with open('python_lessons/assignment3/benzene_hessian.dat', 'r') as f:
  data_benzene = np.array([float(value) for line in f.readlines()[1:] for value in line.split()])
data_benzene

### Reshape the numpy array to a squared matrix: (3N)^2 
hessian_3c1b = data_3c1b.reshape((3*natom,3*natom))
print(hessian_3c1b.shape)
print(hessian_3c1b)

### creat a G matrix
G_matrix = hessian_benzene.copy() #This is to make sure G matrix has the same shape as Hessian matrix, values will be overwritten.
 
for i in range(natom):
  for j in range(natom):
    G_matrix[i*3,j*3]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3,j*3+1]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3,j*3+2]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+1,j*3]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+1,j*3+1]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+1,j*3+2]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+2,j*3]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+2,j*3+1]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))
    G_matrix[i*3+2,j*3+2]=(1/np.sqrt(mass_lib[atoms_list[i]]*mass_lib[atoms_list[j]]))

print(G_matrix.shape)
G_matrix

### multiply the hessian matrix with G matrix to get massweighed hessian matrix
massweighed_hessian = np.multiply(hessian_3c1b,G_matrix)
print(massweighed_hessian)

eigvalue, eigvector = np.linalg.eig(massweighed_hessian)
# print(eigvalue)
# print(eigvector)

eig_format=[abs(float('%.10f'%item)) for item in eigvalue]  #becasue there are negative values
eig_format= sorted(eig_format, reverse=True)
eig_format

amu_to_au = 1/(5.485e-4)
bohr_to_cm = 5.291e-9
c_in_au = 137.036
 
CONSTANT = 1/(2*np.pi*c_in_au) * np.sqrt(1/(amu_to_au * bohr_to_cm * bohr_to_cm))

print(CONSTANT)

frequency = CONSTANT * np.sqrt(eig_format)
frequency